# Building RAG with Qwen2.5

In [ ]:
import json
import os
import shutil
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Step 0: Clear the Chroma database if it exists
persist_dir = "./chroma.db"
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

# Step 1: Folder for JSON Files
input_folder = "Vietnam-Law-rag_json"

# Step 2: Load All JSON Files and Convert to LangChain Documents
documents = []
for file_name in os.listdir(input_folder):
    if file_name.endswith(".json"):
        file_path = os.path.join(input_folder, file_name)
        base_file_name = os.path.splitext(file_name)[0]  # Remove the extension for `file_id`
        
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        
        # Convert JSON data to LangChain Document objects
        documents.extend([
            Document(
                page_content=entry["text"],
                metadata={
                    "id": entry["id"],
                    "article": entry["article"],
                    "clause": entry["clause"],
                    "title": entry["title"],
                    "file_id": base_file_name
                }
            )
            for entry in data
        ])

print(f"Loaded {len(documents)} documents from {input_folder}.")

# Step 3: Initialize HuggingFace Embeddings
embeddings_model = HuggingFaceEmbeddings()

# Step 4: Create Chroma Vector Store
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=persist_dir
)

print("Chroma database created and saved at:", persist_dir)

# Test query
## RAG database builded on cloud servers, fetch them then run the below cell

The aim is to optimize the returned data after the query search before push into the LLM Models, below here use Qwen2.5 for example.

Just download the chroma.db, then symlink or put them in the current working git folder, then run the second cell.

In [ ]:
import torch
from langchain_chroma import Chroma  # Use the updated Chroma import
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings

# Step 1: Load the Chroma Database
persist_dir = "./chroma.db"

# Initialize the embedding function
embeddings_model = HuggingFaceEmbeddings()

# Load the Chroma database with the embedding function
vectorstore = Chroma(
    persist_directory=persist_dir,
    embedding_function=embeddings_model
)

print("Chroma database loaded.")

# Step 2: Load Qwen Model
model_id = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Step 3: Set Device for GPU/CPU
device = 0 if torch.cuda.is_available() else -1

# Step 4: Create a Text-Generation Pipeline with GPU/CPU
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    device=device,
    clean_up_tokenization_spaces=True
)

# Wrap the pipeline for LangChain
hf = HuggingFacePipeline(pipeline=pipe)

print("Model and pipeline initialized.")

In [5]:
import re
from langchain.chains import RetrievalQA

# Move model to GPU
torch.cuda.empty_cache()
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Increase retrieval limit
retriever = vectorstore.as_retriever(search_kwargs={"k": 250})

qa_chain = RetrievalQA.from_chain_type(
    llm=hf,
    retriever=retriever,
    return_source_documents=True
)

def deduplicate_documents(documents):
    """
    Remove duplicate content from the retrieved documents.
    """
    seen_content = set()
    unique_documents = []
    for doc in documents:
        # Use page content as the key for deduplication
        content_key = doc.page_content.strip()
        if content_key not in seen_content:
            seen_content.add(content_key)
            unique_documents.append(doc)
    return unique_documents

def expand_context_with_children(documents):
    """
    Expand parent clauses by appending child clause content using regex to identify parent-child relationships.
    """
    # Organize documents by parent id prefix
    grouped_docs = {}
    for doc in documents:
        # Extract parent ID using regex to match parent prefix (e.g., "Điều 2.33")
        match = re.match(r"(.*?\.\d+)", doc.metadata["id"])
        parent_id = match.group(1) if match else doc.metadata["id"]
        if parent_id not in grouped_docs:
            grouped_docs[parent_id] = []
        grouped_docs[parent_id].append(doc)

    # Create expanded documents
    expanded_documents = []
    for parent_id, docs in grouped_docs.items():
        # Sort documents to ensure children are added in order (e.g., "Điều 2.33a" comes after "Điều 2.33")
        sorted_docs = sorted(docs, key=lambda d: d.metadata["id"])
        # Combine content from parent and children
        combined_text = " ".join(d.page_content for d in sorted_docs)
        # Use the first document's metadata for the combined document
        parent_doc = sorted_docs[0]
        expanded_documents.append({
            "id": parent_doc.metadata["id"],
            "article": parent_doc.metadata.get("article"),
            "clause": parent_doc.metadata.get("clause"),
            "title": parent_doc.metadata.get("title"),
            "text": combined_text,
            "file_id": parent_doc.metadata.get("file_id")
        })

    return expanded_documents

# Query and retrieval
query = "An toàn lao động là gì"
result = qa_chain({"query": query})

# Step 1: Deduplicate retrieved documents
unique_docs = deduplicate_documents(result["source_documents"])

# Step 2: Expand context for parent clauses
expanded_docs = expand_context_with_children(unique_docs)

# Print the Result
print("Answer:", result["result"])

# Print the Source Documents
print("Source Documents:")
for doc in result["source_documents"]:
    print(f"Metadata: {doc.metadata}")
    print(f"Content: {doc.page_content}\n")

# Move model to CPU to release GPU memory
torch.cuda.empty_cache()
model.to("cpu")
torch.cuda.empty_cache()

/tmp/ipykernel_1123/556785003.py:67: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

An toàn, vệ sinh lao động;

An toàn, vệ sinh lao động;

An toàn, vệ sinh lao động;

Quy trình an toàn lao động, vệ sinh lao động.

Kỷ luật lao động, an toàn và vệ sinh lao động;

Bảo đảm làm việc trong điều kiện an toàn lao động, vệ sinh lao động.

Điều tra tai nạn lao động và vi phạm an toàn, vệ sinh lao động.

Tòa án nhân dân có quyền tuyên bố hợp đồng lao động vô hiệu.

Khen thưởng về an toàn, vệ sinh lao động;

Bảo đảm quyền của người lao động được làm việc trong điều kiện an toàn, vệ sinh lao động.

Tranh chấp về an toàn lao động, vệ sinh lao động.

Hợp đồng lao động làm toàn bộ thời gian của các kiểm toán viên hành nghề;

Tổ chức hoạt động kiểm định kỹ thuật an toàn lao động có quyền sau đây:

Tiền công lao động;

Hội đồng Thẩm phán Tòa án nhân dân tối cao;

Tổ chức hoạt động kiểm định kỹ thuật an toàn lao động